In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

### Preprocess the .json

In [95]:
import json
from pprint import pprint

def load_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def merge_json(questions_file, answers_file):
    questions = load_json(questions_file)
    answers = load_json(answers_file)
    
    merged_data = {}
    
    for q_id, q_info in questions.items():
        question_text = q_info['question']
        cot_gt = q_info['explanation (cot)'][0]
        choices = q_info['choices']
        generated_cot = answers[q_id]['explanation (cot)']
        generated_cot = ' '.join(generated_cot).replace('\n', '').strip()
        cop = q_info['answer'][0]
        
        merged_data[question_text] = {
            'generated_cot': generated_cot,
            'cot_gt': cot_gt,
            'opa': choices['0'],
            'opb': choices['1'],
            'opc': choices['2'],
            'opd': choices['3'],
            'cop': int(cop)
        }
    
    return merged_data

questions_file = '../questions_20.json'
answers_file = '../predictions/gpt3.5/few_shot_no_cot_gpt3.5.json'
#answers_file = '../predictions/gpt4/zero_shot_no_cot_gpt4.json'

responses = merge_json(questions_file, answers_file)
print(responses)

{"Which of the following is not true about glomerular capillaries')": {'generated_cot': 'The oncotic pressure of the fluid leaving the glomerular capillaries is greater than that of the fluid entering it because large molecules like proteins are not filtered through the glomerular capillaries.', 'cot_gt': 'Ans-a. The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it Guyton I LpJ1 4-.;anong 23/e p653-6_)Glomerular oncotic pressure (due to plasma protein content) is higher than that of filtrate oncotic pressure in Bowman\'s capsule"Since glucose is freely filtered and the fluid in the Bowman\'s capsule is isotonic with plasma, the concentration of glucose in the filtrate is the same as in the capillaries', 'opa': 'The oncotic pressure of the fluid leaving the capillaries is less than that of fluid entering it', 'opb': 'Glucose concentration in the capillaries is the same as that in glomerular filtrate', 'opc': 'Constriction of afferent aeriole d

In [47]:
from utils import *

In [96]:
# BLEU score
bleu = evaluate_cot(responses, metric="bleu")
bleu

# ROUGE score
# rouge = evaluate_cot(responses, metric="rouge")
# rouge

{'bleu': 0.016510923692680632,
 'precisions': [0.4608355091383812,
  0.1380697050938338,
  0.06611570247933884,
  0.03541076487252125],
 'brevity_penalty': 0.14945150951762787,
 'length_ratio': 0.34473447344734476,
 'translation_length': 766,
 'reference_length': 2222}

In [97]:
# SentenceBERT score
scb_recall, scb_score = evaluate_cot(responses, metric="sentencebert")
scb_recall, scb_score

(1.0, 0.66327727)

In [4]:
# This takes a long time
evaluate_cot(responses, metric="kgbased_v2")

...initializing KG
...start loading the KG
